In [116]:
from tf_pwa.config_loader import ConfigLoader
from tf_pwa.amp.core import Particle
import numpy as np
import tensorflow as tf
import json
import warnings

In [33]:
config = ConfigLoader("config_a.yml")
config.set_params("final_params.json")

phsp = config.get_phsp_noeff()


/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/amp/core.py:138: UserWarning: No model named C(BWR_LS) found, use default instead.
  warnings.warn(
/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/amp/core.py:138: UserWarning: No model named C(BWR) found, use default instead.
  warnings.warn(
/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/amp/core.py:138: UserWarning: No model named C(New) found, use default instead.
  warnings.warn(
/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/amp/core.py:138: UserWarning: No model named C(one) found, use default instead.
  warnings.warn(
/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/amp/core.py:138: UserWarning: No model named C2(BWR) found, use default instead.
  warnings.warn(
/home/akazatsky/Desktop/CompProject/tf-pwa/tf_pwa/config_loader/config_loader.py:1035: UserWarning: Neglect ['X(3872)_mass', 'X(3872)_width', 'chi(c2)(3930)_mass', 'chi(c2)(3930)_width', 'Psi(4040)_mass', 'Psi(4040)_width', 'X0(2900)_mass', 'X0(2900)_width', 'X

In [ ]:
"""
Functions
"""

def print_structure(d, indent=0):
    prefix = "  " * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f"{prefix}{k}:")
            print_structure(v, indent + 1)
    elif isinstance(d, tf.Tensor):
        print(f"{prefix}tf.Tensor(shape={d.shape}, dtype={d.dtype})")
    elif isinstance(d, np.ndarray):
        print(f"{prefix}np.ndarray(shape={d.shape}, dtype={d.dtype})")
    else:
        print(f"{prefix}{type(d)}")


def extract_single_event(data):
    if isinstance(data, dict):
        return {k: extract_single_event(v) for k, v in data.items()}
    elif isinstance(data, (tf.Tensor, np.ndarray)):
        return data[:1]
    else:
        return data
    
    
def complex_amplitude(single_event):
    return config.get_amplitude().decay_group.get_amp(single_event).numpy().flatten()[0]


def show_resonances(config):

    warnings.filterwarnings("ignore")

    decay_chains = config.get_decay()

    used_res = set()

    for chain in decay_chains:
        for decay in chain:
            if hasattr(decay, "core") and hasattr(decay.core, "decay"):
                used_res.add(decay.core.name)
            if hasattr(decay, "mother"):
                used_res.add(decay.mother.name)

    return sorted(used_res)

In [134]:
"Filtering out one event"
phsp_single = extract_single_event(phsp)

print_structure(phsp_single)

particle:
  pi:
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  (D0, K, pi):
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  (D0, pi):
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  (D, D0, pi):
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  (D, K):
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  D0:
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  K:
    m:
      tf.Tensor(shape=(1,), dtype=<dtype: 'float64'>)
    p:
      tf.Tensor(shape=(1, 4), dtype=<dtype: 'float64'>)
  Bp:
    m:
      tf.Tensor(shape=(1,), dtype=<dtype

In [135]:
"Calculating the complex amplitude"
A_single = complex_amplitude(phsp_single)

print(A_single)

(-0.18786023654530792-0.2787192355641921j)


In [136]:
"Display all resonances involved"
Used_resonances = show_resonances(config)

for i in Used_resonances:
    print(i)

Bp
Dst
NR(0-)SPm
NR(0-)SPp
NR(1+)PSp
NR(1-)PPm
Psi(4040)
X(3872)
X(3915)(0-)
X(3940)(1+)
X(3993)
X(4300)
X0(2900)
X1(2900)
chi(c2)(3930)
